# Theoretical Option Pricing

# 1. Introduction


In the realm of options pricing, traditional models often simplify the intricate dynamics of underlying stock movements or are limited to certain constraints.



### Dynamic Programming

In this project we approach theoretical option pricing as a dynamic programming problem in which at each time step we have information $s_t \in \mathcal{S}$ such as the underlying price, time till expiry, interest rates, etc. We also have a set of actions we can take at each time step $a \in \mathcal{A}$ such as early exercise, hedging adjustment, taking no action, etc. We also have functions defining the cost/reward of taking an action $C(s_t, a_t)$ and probabilty of transitioning from one state to another $P(s_{t + 1} ~|~ s_t, a_t)$ typically independent of $a_t$ and corresponds to price movements of the underlying. Our goal therefore is to find $V^*(s_t)$ which denotes the theoretical value of an option under the assumtion that the holder of the contract takes the most optional actions during the lifetime of the contract. By the Bellman Optimality Principle, $V^*(s_t)$ can be recursively defined as:

$$
\begin{equation}
V^*(s_t) = 
    \max_{a_t \in \mathcal{A}} 
    \left\{
        C(s_t, a_t) + \int_{s_{t+1}} P(s_{t + 1} ~|~ s_t, a_t) V^*(s_{t + 1})  
    \right\}
\end{equation}
$$




### Organization
The rest of this notebook is organized as follows:

- In Section 2 we will model our underlying assets price dynamics using the Bates Model.
- In Section 3 we will describe the algorithm we will use to approximate $V^*(s_t)$.
- In Section 4 we will provide dynamic programming formualtions for the different contracts we will be considering.
- In Section 5 we will fit the models and visualize the results.

# 2. Bates Model

In this project we assume the S&P 500 follows the Heston model which is a Stochastic Volatility model defined by the following stochastic differential equations:

$$ 
\begin{align}
dS_t &= \mu S_t ~ dt + \sqrt{V_t} S_t ~ dW_{S,t}
\\
dV_t &= \kappa (\theta - V_t) ~ dt + \sigma \sqrt{V_t} ~ dW_{V,t}
\end{align}
$$ 

where:
- $S_t$ represents the stock price at time $t$,
- $V_t$ denotes the stochastic volatility process,
- $\mu$ is the drift rate,
- $\kappa$ is the mean-reversion rate of volatility,
- $\theta$ is the long-term average volatility,
- $\sigma$ is the volatility of volatility,
- $dW_{S,t}$ and $dW_{V,t}$ are Wiener processes for the stock price and volatility, respectively,


# 3. Experiments

## American Option

#### Dynamic Programming Formulation

Given a strike price $K$ our option's pay off after exercising at time $t$ is $\max\{S - K, 0\}$. Further more since this is an American call option, we can exercise at any time $t \ge 0$. Therefore at each time step we have the choice of exercising and recieving a payoff of $\max\{0, S - K\}$ or not exercising. If we dont exercise, our underlying stock and variance processes move to new values $S_{t + dt}$ and $V_{t + dt}$ and we have a new decision to make of whether or not we should exercise. This recursive relationship can be modeled as follows:

$$
\begin{align}
V_0(S_t) &= \max\{S_t - K, 0\} \\
V_{t > 0}(S_t) &= \max\left\{ \max\{S_t - K, 0\}, \mathbb{E}\bigg[V_{t - dt}(S_{t + dt})\bigg]\right\}
\end{align}
$$

and in the case of a put option:

$$
\begin{align}
V_0(S_t) &= \max\{K - S_t, 0\} \\
V_{t > 0}(S_t) &= \max\left\{ \max\{K - S_t, 0\}, \mathbb{E}\bigg[V_{t - dt}(S_{t + dt})\bigg]\right\}
\end{align}
$$


In [ ]:
from engines import AmericanVanilla

model = AmericanVanilla(
    underlying=underlying, 
    strike=100, 
    call=True, 
    M0=100,
    M1=100
)

In [22]:
import numpy as np

np.set_printoptions(suppress=True)
points = np.random.uniform([0, 0], [100, 0.1], size=(10, 2)).T
print(points)

# vecs = [points[:, i] for i in range(2)]

# print(vecs)

[[57.45074818  1.02738845 71.45590771 48.80255421 87.04682614 16.63802432
  83.01613259 94.31942755 67.50510094 45.07845484]
 [ 0.01139361  0.04460803  0.03545758  0.07622961  0.07685106  0.09206857
   0.02658116  0.04650445  0.05375785  0.09157377]]


In [7]:
import numpy as np

x = np.arange(start=1, stop=100)


def running_arithmetic_mean(X):
    mu = np.zeros(len(X))
    mu[0] = X[0]

    for t in range(1, len(X)):
        mu[t] = (t * mu[t-1] + X[t]) / (t + 1)

    return mu

def running_geometric_mean(X):
    mu = np.zeros(len(X))
    mu[0] = X[0]

    for t in range(1, len(X)):
        mu[t] = (X[t] * mu[t-1] ** t) ** (1 / (t + 1))

    return mu

print(running_arithmetic_mean(x))
print(running_geometric_mean(x))

[ 1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5  7.   7.5
  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5 14.  14.5
 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5 20.  20.5 21.  21.5
 22.  22.5 23.  23.5 24.  24.5 25.  25.5 26.  26.5 27.  27.5 28.  28.5
 29.  29.5 30.  30.5 31.  31.5 32.  32.5 33.  33.5 34.  34.5 35.  35.5
 36.  36.5 37.  37.5 38.  38.5 39.  39.5 40.  40.5 41.  41.5 42.  42.5
 43.  43.5 44.  44.5 45.  45.5 46.  46.5 47.  47.5 48.  48.5 49.  49.5
 50. ]
[ 1.          1.41421356  1.81712059  2.21336384  2.60517108  2.99379517
  3.38001516  3.7643506   4.14716627  4.52872869  4.90923878  5.28885199
  5.66769118  6.04585517  6.42342475  6.8004668   7.17703736  7.55318386
  7.92894684  8.3043612   8.67945723  9.05426137  9.42879687  9.80308428
 10.17714183 10.55098582 10.92463083 11.29808998 11.67137514 12.04449704
 12.41746545 12.79028928 13.16297668 13.5355351  13.90797143 14.28029199
 14.65250261 15.02460871 15.39661529 15.76852702 16.140348